# 🚀 Phase 3: Traffic Forecasting Models

## NASA HTTP Log Data - Autoscaling Competition

**Mục tiêu:**
- Train/Test Split: Train (Jul 1 - Aug 22), Test (Aug 23 - Aug 31)
- Dự báo: `request_count` và `total_bytes`
- Models: **Prophet** + **XGBoost**
- Metrics: RMSE, MAE, MAPE

## 1. Setup & Imports

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# Time Series
from prophet import Prophet

# Config
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)

print("✅ All libraries imported successfully!")

## 2. Load Data

In [ ]:
# Load 5-minute aggregated data
DATA_PATH = "../processed_data/nasa_traffic_5m.csv"

df = pd.read_csv(DATA_PATH, parse_dates=['timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize(None)  # Remove timezone

print(f"📊 Dataset shape: {df.shape}")
print(f"📅 Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"\n🔍 Columns: {df.columns.tolist()}")
df.head()

In [ ]:
# Check for missing data gap (Hurricane: Aug 1 14:52 - Aug 3 04:36)
df = df.set_index('timestamp').sort_index()

# Visualize data availability
fig = px.scatter(df.reset_index(), x='timestamp', y='request_count', 
                 title='📈 Request Count Over Time (Note the gap in early August)')
fig.update_traces(marker=dict(size=2))
fig.show()

## 3. Train/Test Split

**Theo yêu cầu cuộc thi:**
- **Train Set:** July 1 → August 22, 1995
- **Test Set:** August 23 → August 31, 1995

In [ ]:
# Define split date
SPLIT_DATE = '1995-08-23'

# Split data
train_df = df[df.index < SPLIT_DATE].copy()
test_df = df[df.index >= SPLIT_DATE].copy()

print(f"📊 Train set: {len(train_df)} samples")
print(f"   From: {train_df.index.min()} to {train_df.index.max()}")
print(f"\n📊 Test set: {len(test_df)} samples") 
print(f"   From: {test_df.index.min()} to {test_df.index.max()}")

## 4. Feature Engineering

Tạo các features cho XGBoost:
- **Time features:** hour, day_of_week, day_of_month, is_weekend
- **Lag features:** lag_1, lag_2, lag_3, lag_12 (1h), lag_288 (1 day @ 5min interval)

In [ ]:
def create_features(df):
    """Create time-based and lag features for ML model"""
    df = df.copy()
    
    # Time features
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['day_of_month'] = df.index.day
    df['is_weekend'] = (df.index.dayofweek >= 5).astype(int)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    
    # Lag features for request_count
    for lag in [1, 2, 3, 6, 12, 288]:  # 5min, 10min, 15min, 30min, 1h, 1day
        df[f'request_lag_{lag}'] = df['request_count'].shift(lag)
        df[f'bytes_lag_{lag}'] = df['total_bytes'].shift(lag)
    
    # Rolling statistics (1 hour = 12 intervals of 5min)
    df['request_rolling_mean_1h'] = df['request_count'].rolling(12).mean()
    df['request_rolling_std_1h'] = df['request_count'].rolling(12).std()
    df['bytes_rolling_mean_1h'] = df['total_bytes'].rolling(12).mean()
    
    return df

# Apply feature engineering to full dataset first, then split
df_features = create_features(df)

# Re-split after feature engineering
train_features = df_features[df_features.index < SPLIT_DATE].copy()
test_features = df_features[df_features.index >= SPLIT_DATE].copy()

# Drop NaN rows (from lag features)
train_features = train_features.dropna()
test_features = test_features.dropna()

print(f"✅ Features created!")
print(f"   Train: {len(train_features)} samples")
print(f"   Test: {len(test_features)} samples")
print(f"\n📋 Feature columns: {[c for c in train_features.columns if c not in df.columns]}")

## 5. Evaluation Metrics

In [ ]:
def calculate_metrics(y_true, y_pred, model_name="Model"):
    """Calculate RMSE, MAE, MAPE"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    # MAPE - avoid division by zero
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    
    print(f"\n📊 {model_name} Performance:")
    print(f"   RMSE: {rmse:,.2f}")
    print(f"   MAE:  {mae:,.2f}")
    print(f"   MAPE: {mape:.2f}%")
    
    return {'rmse': rmse, 'mae': mae, 'mape': mape}

---
# 🔮 MODEL 1: Facebook Prophet

Prophet là model time-series của Facebook, tốt cho:
- Dữ liệu có seasonality (daily, weekly)
- Xử lý missing data tự động
- Dễ tune hyperparameters

### 6.1 Prophet - Request Count Prediction

In [ ]:
# Prepare data for Prophet (requires 'ds' and 'y' columns)
prophet_train_requests = train_df.reset_index()[['timestamp', 'request_count']].rename(
    columns={'timestamp': 'ds', 'request_count': 'y'}
)

# Initialize and train Prophet model
print("🔄 Training Prophet model for Request Count...")
prophet_requests = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=False,  # Only 2 months of data
    changepoint_prior_scale=0.05,  # Regularization
    seasonality_mode='multiplicative'
)
prophet_requests.fit(prophet_train_requests)
print("✅ Prophet model trained!")

In [ ]:
# Predict on test set
prophet_test_requests = test_df.reset_index()[['timestamp']].rename(columns={'timestamp': 'ds'})
prophet_pred_requests = prophet_requests.predict(prophet_test_requests)

# Get predictions
y_true_requests = test_df['request_count'].values
y_pred_requests_prophet = prophet_pred_requests['yhat'].values

# Evaluate
prophet_request_metrics = calculate_metrics(
    y_true_requests, 
    y_pred_requests_prophet, 
    "Prophet - Request Count"
)

### 6.2 Prophet - Total Bytes Prediction

In [ ]:
# Prepare data for Prophet - Total Bytes
prophet_train_bytes = train_df.reset_index()[['timestamp', 'total_bytes']].rename(
    columns={'timestamp': 'ds', 'total_bytes': 'y'}
)

# Train Prophet model for bytes
print("🔄 Training Prophet model for Total Bytes...")
prophet_bytes = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=False,
    changepoint_prior_scale=0.05,
    seasonality_mode='multiplicative'
)
prophet_bytes.fit(prophet_train_bytes)

# Predict
prophet_pred_bytes = prophet_bytes.predict(prophet_test_requests)

# Evaluate
y_true_bytes = test_df['total_bytes'].values
y_pred_bytes_prophet = prophet_pred_bytes['yhat'].values

prophet_bytes_metrics = calculate_metrics(
    y_true_bytes, 
    y_pred_bytes_prophet, 
    "Prophet - Total Bytes"
)

---
# 🌲 MODEL 2: XGBoost

XGBoost là gradient boosting model mạnh cho tabular data:
- Sử dụng lag features và time features
- Nhanh và hiệu quả
- Dễ interpret feature importance

### 7.1 XGBoost - Request Count Prediction

In [ ]:
# Define feature columns (exclude target and original columns)
feature_cols = [col for col in train_features.columns if col not in [
    'request_count', 'total_bytes', 'status_2xx', 'status_3xx', 
    'status_4xx', 'status_5xx', 'is_outage'
]]

print(f"📋 Features used: {feature_cols}")

# Prepare train/test data
X_train = train_features[feature_cols]
y_train_requests = train_features['request_count']
y_train_bytes = train_features['total_bytes']

X_test = test_features[feature_cols]
y_test_requests = test_features['request_count']
y_test_bytes = test_features['total_bytes']

print(f"\n✅ X_train shape: {X_train.shape}")
print(f"✅ X_test shape: {X_test.shape}")

In [ ]:
# Train XGBoost for Request Count
print("🔄 Training XGBoost model for Request Count...")

xgb_requests = xgb.XGBRegressor(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

xgb_requests.fit(X_train, y_train_requests)
print("✅ XGBoost model trained!")

# Predict
y_pred_requests_xgb = xgb_requests.predict(X_test)

# Evaluate
xgb_request_metrics = calculate_metrics(
    y_test_requests.values, 
    y_pred_requests_xgb, 
    "XGBoost - Request Count"
)

### 7.2 XGBoost - Total Bytes Prediction

In [ ]:
# Train XGBoost for Total Bytes
print("🔄 Training XGBoost model for Total Bytes...")

xgb_bytes = xgb.XGBRegressor(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

xgb_bytes.fit(X_train, y_train_bytes)

# Predict
y_pred_bytes_xgb = xgb_bytes.predict(X_test)

# Evaluate
xgb_bytes_metrics = calculate_metrics(
    y_test_bytes.values, 
    y_pred_bytes_xgb, 
    "XGBoost - Total Bytes"
)

### 7.3 XGBoost Feature Importance

In [ ]:
# Feature importance for Request Count model
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_requests.feature_importances_
}).sort_values('importance', ascending=False)

fig = px.bar(importance_df.head(15), x='importance', y='feature', 
             orientation='h', title='🔍 Top 15 Feature Importance (XGBoost - Request Count)')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

---
## 8. Model Comparison & Summary

In [ ]:
# Create comparison table
comparison_data = {
    'Model': ['Prophet', 'Prophet', 'XGBoost', 'XGBoost'],
    'Target': ['Request Count', 'Total Bytes', 'Request Count', 'Total Bytes'],
    'RMSE': [
        prophet_request_metrics['rmse'],
        prophet_bytes_metrics['rmse'],
        xgb_request_metrics['rmse'],
        xgb_bytes_metrics['rmse']
    ],
    'MAE': [
        prophet_request_metrics['mae'],
        prophet_bytes_metrics['mae'],
        xgb_request_metrics['mae'],
        xgb_bytes_metrics['mae']
    ],
    'MAPE (%)': [
        prophet_request_metrics['mape'],
        prophet_bytes_metrics['mape'],
        xgb_request_metrics['mape'],
        xgb_bytes_metrics['mape']
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("📊 MODEL COMPARISON - Test Set (Aug 23 - Aug 31, 1995)")
print("="*70)
comparison_df

---
## 9. Visualization: Actual vs Predicted

In [ ]:
# Visualization: Request Count - Actual vs Predicted
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=('Prophet - Request Count', 'XGBoost - Request Count'),
                    vertical_spacing=0.12)

# Prophet predictions
fig.add_trace(
    go.Scatter(x=test_df.index, y=y_true_requests, name='Actual', 
               line=dict(color='blue', width=1)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=test_df.index, y=y_pred_requests_prophet, name='Prophet Predicted',
               line=dict(color='red', width=1, dash='dash')),
    row=1, col=1
)

# XGBoost predictions  
fig.add_trace(
    go.Scatter(x=test_features.index, y=y_test_requests, name='Actual',
               line=dict(color='blue', width=1), showlegend=False),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=test_features.index, y=y_pred_requests_xgb, name='XGBoost Predicted',
               line=dict(color='green', width=1, dash='dash')),
    row=2, col=1
)

fig.update_layout(height=700, title_text="📈 Request Count: Actual vs Predicted (Test Set)")
fig.show()

In [ ]:
# Visualization: Total Bytes - Actual vs Predicted
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=('Prophet - Total Bytes', 'XGBoost - Total Bytes'),
                    vertical_spacing=0.12)

# Prophet predictions
fig.add_trace(
    go.Scatter(x=test_df.index, y=y_true_bytes, name='Actual', 
               line=dict(color='blue', width=1)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=test_df.index, y=y_pred_bytes_prophet, name='Prophet Predicted',
               line=dict(color='red', width=1, dash='dash')),
    row=1, col=1
)

# XGBoost predictions  
fig.add_trace(
    go.Scatter(x=test_features.index, y=y_test_bytes, name='Actual',
               line=dict(color='blue', width=1), showlegend=False),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=test_features.index, y=y_pred_bytes_xgb, name='XGBoost Predicted',
               line=dict(color='green', width=1, dash='dash')),
    row=2, col=1
)

fig.update_layout(height=700, title_text="📊 Total Bytes: Actual vs Predicted (Test Set)")
fig.show()

---
## 10. Save Models

In [ ]:
import pickle
import json
from datetime import datetime

# Save models
MODEL_DIR = "../saved_models"

# Save Prophet models
with open(f"{MODEL_DIR}/prophet_requests.pkl", "wb") as f:
    pickle.dump(prophet_requests, f)
    
with open(f"{MODEL_DIR}/prophet_bytes.pkl", "wb") as f:
    pickle.dump(prophet_bytes, f)

# Save XGBoost models  
xgb_requests.save_model(f"{MODEL_DIR}/xgb_requests.json")
xgb_bytes.save_model(f"{MODEL_DIR}/xgb_bytes.json")

# Save metrics
metrics_summary = {
    'trained_at': datetime.now().isoformat(),
    'train_period': 'July 1 - August 22, 1995',
    'test_period': 'August 23 - August 31, 1995',
    'models': {
        'prophet_requests': prophet_request_metrics,
        'prophet_bytes': prophet_bytes_metrics,
        'xgb_requests': xgb_request_metrics,
        'xgb_bytes': xgb_bytes_metrics
    }
}

with open(f"{MODEL_DIR}/metrics_summary.json", "w") as f:
    json.dump(metrics_summary, f, indent=2)

print("✅ All models saved to ../saved_models/")
print("   - prophet_requests.pkl")
print("   - prophet_bytes.pkl")
print("   - xgb_requests.json")
print("   - xgb_bytes.json")
print("   - metrics_summary.json")

---
## 11. Phase 4: Autoscaling Simulation

Sử dụng dự báo để simulate autoscaling với cooldown/hysteresis.

In [ ]:
def simulate_autoscaling(predicted_loads, 
                         capacity_per_server=500,  # requests per 5min per server
                         scale_up_threshold=0.8,
                         scale_down_threshold=0.3,
                         cooldown_periods=6,  # 6 x 5min = 30min cooldown
                         min_servers=1,
                         max_servers=20):
    """
    Simulate autoscaling based on predicted traffic.
    
    Parameters:
    -----------
    predicted_loads: array of predicted request counts
    capacity_per_server: max requests one server can handle per interval
    scale_up_threshold: utilization % to trigger scale up (default 80%)
    scale_down_threshold: utilization % to trigger scale down (default 30%)
    cooldown_periods: number of intervals to wait between scaling actions
    
    Returns:
    --------
    DataFrame with scaling decisions
    """
    
    results = []
    current_servers = 2  # Start with 2 servers
    last_scale_time = -cooldown_periods  # Allow immediate first scaling
    
    for i, load in enumerate(predicted_loads):
        capacity = current_servers * capacity_per_server
        utilization = load / capacity if capacity > 0 else 1.0
        
        action = "maintain"
        reason = ""
        
        # Check if cooldown has passed
        if i - last_scale_time >= cooldown_periods:
            if utilization > scale_up_threshold:
                # Scale up
                needed_servers = int(np.ceil(load / (capacity_per_server * 0.7)))  # Target 70% util
                new_servers = min(max_servers, max(current_servers + 1, needed_servers))
                if new_servers > current_servers:
                    action = "scale_up"
                    reason = f"Utilization {utilization:.1%} > {scale_up_threshold:.0%}"
                    current_servers = new_servers
                    last_scale_time = i
                    
            elif utilization < scale_down_threshold:
                # Scale down
                new_servers = max(min_servers, current_servers - 1)
                if new_servers < current_servers:
                    action = "scale_down"
                    reason = f"Utilization {utilization:.1%} < {scale_down_threshold:.0%}"
                    current_servers = new_servers
                    last_scale_time = i
        else:
            reason = f"Cooldown ({cooldown_periods - (i - last_scale_time)} periods left)"
        
        results.append({
            'period': i,
            'predicted_load': load,
            'servers': current_servers,
            'capacity': current_servers * capacity_per_server,
            'utilization': utilization,
            'action': action,
            'reason': reason
        })
    
    return pd.DataFrame(results)

# Run simulation with XGBoost predictions
scaling_results = simulate_autoscaling(y_pred_requests_xgb)
print("🖥️ Autoscaling Simulation Results:")
print(f"   Total periods: {len(scaling_results)}")
print(f"   Scale up events: {(scaling_results['action'] == 'scale_up').sum()}")
print(f"   Scale down events: {(scaling_results['action'] == 'scale_down').sum()}")
print(f"   Server range: {scaling_results['servers'].min()} - {scaling_results['servers'].max()}")

In [ ]:
# Visualize autoscaling simulation
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=('Predicted Load & Server Capacity', 'Number of Active Servers'),
                    vertical_spacing=0.15)

# Load vs Capacity
fig.add_trace(
    go.Scatter(y=scaling_results['predicted_load'], name='Predicted Load',
               line=dict(color='blue')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(y=scaling_results['capacity'], name='Server Capacity',
               line=dict(color='green', dash='dash')),
    row=1, col=1
)

# Number of servers
fig.add_trace(
    go.Scatter(y=scaling_results['servers'], name='Active Servers',
               line=dict(color='orange'), fill='tozeroy'),
    row=2, col=1
)

# Mark scale events
scale_up_idx = scaling_results[scaling_results['action'] == 'scale_up'].index
scale_down_idx = scaling_results[scaling_results['action'] == 'scale_down'].index

fig.add_trace(
    go.Scatter(x=scale_up_idx, y=scaling_results.loc[scale_up_idx, 'servers'],
               mode='markers', name='Scale Up', marker=dict(color='red', size=10, symbol='triangle-up')),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=scale_down_idx, y=scaling_results.loc[scale_down_idx, 'servers'],
               mode='markers', name='Scale Down', marker=dict(color='blue', size=10, symbol='triangle-down')),
    row=2, col=1
)

fig.update_layout(height=600, title_text="🖥️ Autoscaling Simulation (with Cooldown)")
fig.show()

## ✅ Summary

### Models Trained:
1. **Prophet** - Time series model với daily/weekly seasonality
2. **XGBoost** - Gradient boosting với lag features

### Key Results:
- Train: July 1 - August 22, 1995
- Test: August 23 - August 31, 1995
- Evaluated on: Request Count & Total Bytes
- Metrics: RMSE, MAE, MAPE

### Next Steps:
1. Fine-tune hyperparameters
2. Try ensemble methods
3. Integrate with live dashboard
4. Deploy autoscaling logic